In [1]:
import os
import pandas as pd

In [2]:
while "notebooks" in os.getcwd():
    os.chdir("../")

In [3]:
os.chdir("src")

In [4]:
from decision_tree.tree import parse_xgboost_dump, TreeEnsemble

In [5]:
trees = parse_xgboost_dump("../models/heloc-scaled-gbdt_dumped.txt")

In [6]:
some_trees = TreeEnsemble(trees.trees[0:50])

In [7]:
df = pd.read_csv("../data/heloc-scaled-test.csv")

In [8]:
df

,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,MaxDelq2PublicRecLast12M,...,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance,RiskPerformance
0,-0.214900,1.699313,-0.056723,2.459516,-0.363267,1.201020,1.712719,-1.564641,-1.466360,-3.498869,...,-0.488720,-0.211494,-0.187532,-1.005586,0.090931,-0.702250,-0.955119,-0.062510,-1.379531,0.0
1,1.716386,-0.361454,-0.280331,0.499039,-0.185539,-0.476838,-0.394914,0.646149,0.946849,0.755470,...,0.256357,-0.668264,-0.652592,-0.866162,-0.122189,-0.029287,-0.307994,-0.062510,0.391373,0.0
2,0.903213,0.638018,-0.578475,-0.027656,2.036062,-0.476838,-0.394914,0.646149,0.946849,0.755470,...,1.852951,1.615589,1.672709,-1.075297,1.598802,-0.365768,-0.307994,-0.740481,-1.879017,1.0
3,-0.011607,-0.309935,-0.503939,-1.139568,-0.896451,-0.476838,-0.394914,0.646149,0.946849,0.755470,...,-0.701599,0.245277,0.277528,0.946344,1.446950,0.307195,-0.307994,0.615461,0.981674,0.0
4,0.394979,0.586499,-0.503939,0.323474,1.680606,1.201020,0.658902,-0.119124,0.681985,0.147707,...,-0.062962,-0.668264,-0.652592,-0.238756,-0.096058,1.316640,-0.307994,-0.062510,-0.743822,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970,-0.926427,-1.731864,-0.429403,-1.871090,-0.896451,-0.476838,-0.394914,-0.884398,-1.495790,-1.067818,...,-0.169401,-0.668264,-0.652592,-0.761594,0.090931,-0.029287,-0.955119,-0.740481,-0.743822,0.0
1971,-0.621487,-0.608746,-0.056723,-0.964003,-1.429635,0.362091,0.658902,-0.799368,-0.377473,0.147707,...,-0.275841,0.702048,0.742588,0.493218,0.090931,-0.365768,0.056953,-0.062510,1.526568,0.0
1972,0.191686,-0.598442,-0.429403,-0.612873,-0.096675,-0.476838,-0.394914,0.646149,0.946849,0.755470,...,1.852951,1.158818,1.207648,-0.308468,-1.438241,0.307195,0.339132,0.615461,-0.425967,0.0
1973,1.309799,0.205257,-0.578475,-0.964003,3.635614,-0.476838,-0.394914,0.646149,0.946849,0.755470,...,-0.701599,0.245277,0.277528,-1.214721,-0.096058,-0.029287,0.339132,-0.740481,-2.060648,1.0


In [9]:
x = df.iloc[0]

In [10]:
x

ExternalRiskEstimate                 -0.214900
MSinceOldestTradeOpen                 1.699313
MSinceMostRecentTradeOpen            -0.056723
AverageMInFile                        2.459516
NumSatisfactoryTrades                -0.363267
NumTrades60Ever2DerogPubRec           1.201020
NumTrades90Ever2DerogPubRec           1.712719
PercentTradesNeverDelq               -1.564641
MSinceMostRecentDelq                 -1.466360
MaxDelq2PublicRecLast12M             -3.498869
MaxDelqEver                          -2.376086
NumTotalTrades                       -0.280830
NumTradesOpeninLast12M                0.082920
PercentInstallTrades                 -1.310432
MSinceMostRecentInqexcl7days         -0.488720
NumInqLast6M                         -0.211494
NumInqLast6Mexcl7days                -0.187532
NetFractionRevolvingBurden           -1.005586
NetFractionInstallBurden              0.090931
NumRevolvingTradesWBalance           -0.702250
NumInstallTradesWBalance             -0.955119
NumBank2NatlT

In [11]:
y = some_trees.eval(x)

In [12]:
y

-0.016586913932000003

In [13]:
from decision_tree.prediction_gap import NormalPredictionGap

In [14]:
pred_gap = NormalPredictionGap(10)

Powinno wyjść 0

In [15]:
pred_gap.prediction_gap_fixed(some_trees, x, y, set([]))

-1.1689054672109345e-19

PGI dla każdego pojedynczego featura

In [16]:
for column in df.columns:
    print("%-40s %.09f %.09f %.09f" % (column,
        pred_gap.prediction_gap_fixed(some_trees, x, y, set([column])),
        pred_gap.prediction_gap_single_squared(some_trees, x, column, y),
        pred_gap.prediction_gap_single_abs(some_trees, x, column, y)
    ))

ExternalRiskEstimate                     0.000000001 0.000000001 0.000022742
MSinceOldestTradeOpen                    -0.000000000 0.000000000 0.000000000
MSinceMostRecentTradeOpen                -0.000000000 0.000000000 0.000000000
AverageMInFile                           -0.000000000 0.000000000 0.000000000
NumSatisfactoryTrades                    -0.000000000 0.000000000 0.000000000
NumTrades60Ever2DerogPubRec              -0.000000000 0.000000000 0.000000000
NumTrades90Ever2DerogPubRec              -0.000000000 0.000000000 0.000000000
PercentTradesNeverDelq                   -0.000000000 0.000000000 0.000000000
MSinceMostRecentDelq                     -0.000000000 0.000000000 0.000000000
MaxDelq2PublicRecLast12M                 -0.000000000 0.000000000 0.000000000
MaxDelqEver                              -0.000000000 0.000000000 0.000000000
NumTotalTrades                           -0.000000000 0.000000000 0.000000000
NumTradesOpeninLast12M                   -0.000000000 0.000000000

Poniżej przykład przeliczony na kartce.

In [17]:
from decision_tree.tree import Split, Leaf

In [18]:
test_tree = TreeEnsemble([Split("dupa", 0, Split("dupa", -0.5, Leaf(-1), Leaf(5), Leaf(0)), Leaf(1), Leaf(0))])

In [19]:
x = {'dupa': 0.1}
y = test_tree.eval(x)

In [20]:
pred_gap.prediction_gap_fixed(test_tree, x, y, set(["dupa"]))

2.2232364908150153

In [21]:
pred_gap.prediction_gap_single_squared(test_tree, x, "dupa", y)

2.2232364908151765